# データ空間の回転系手法まとめ（PCA・因子分析・ICA）

本ノートブックでは、多変量データの次元削減や構造抽出において「回転（Rotation）」や「基底変換」の概念が重要となる3つの主要な手法、**主成分分析 (PCA)**、**因子分析 (Factor Analysis)**、**独立成分分析 (ICA)** について、その理論的背景と実際の挙動を比較します。

## 目的と位置づけ
- **PCA（主成分分析）**：分散を最大化する直交基底を見つけ、次元削減・可視化・ノイズ除去に使う。
- **因子分析（FA）**：観測変数を少数の潜在因子＋誤差で説明するモデル。説明可能性と因子解釈が目的。
- **ICA（独立成分分析）**：観測信号を統計的に独立な成分に分解する。混合信号の分離や特徴抽出に使う。

## 直感的な違い（ざっくり）
| 手法 | 目的 | 仮定 | 得られる成分 | 典型用途 |
|---|---|---|---|---|
| **PCA** | 分散最大化 | 線形・直交・ガウス分布（暗黙） | 直交な主成分（無相関） | 次元削減、可視化、前処理 |
| **因子分析** | 潜在因子の推定 | 線形・誤差分離・因子負荷 | 潜在因子（回転により解釈容易化） | 心理尺度、アンケート分析、概念抽出 |
| **ICA** | 独立成分の抽出 | 線形・非ガウス・独立 | 独立成分（非直交可） | 信号分離（カクテルパーティー問題）、脳波解析 |

## 何が「回転」か
- **PCA**: データの分散が最大になる方向へ座標軸を**直交回転**させます。
- **因子分析**: 抽出された因子の解釈を容易にするため、バリマックス回転（Varimax）などの**回転**操作を行うことが標準的です。
- **ICA**: 信号の独立性が最大になるような座標系（必ずしも直交しない）を探す操作であり、これも広義の**回転**とスケーリングを含みます。

## インタラクティブ・デモンストレーション

以下のツールを使用して、混合行列（Mixing Matrix）の回転角度やノイズレベルを変更し、
観測される混合信号の分布形状と、それに対するPCAおよびICAの分離結果がどのように変化するかを観察してください。

- **混合信号**: 原信号が線形変換されたもの。角度を変えると相関の強さが変わります。
- **PCA**: データの分散が最大になる直交軸を探します。分布がガウス分布に近い場合（正弦波など）は有効ですが、独立性までは保証しません。
- **ICA**: 非ガウス性を最大化する方向を探し、元の独立した信号（ノコギリ波など）を復元しようとします。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, FactorAnalysis, FastICA
from scipy import signal
import ipywidgets as widgets
from IPython.display import display, clear_output

# 日本語フォントの設定
import matplotlib.font_manager as fm
font_path = '../fonts/ipaexg.ttf'
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'IPAexGothic'

%matplotlib inline

def run_interactive_demo():
    # 1. 信号の生成 (Sources)
    n_samples = 2000
    time = np.linspace(0, 8, n_samples)
    s1 = np.sin(2 * time)  # 正弦波 (Gaussian-like)
    s2 = signal.sawtooth(2 * np.pi * time)  # ノコギリ波 (Non-Gaussian)
    S = np.c_[s1, s2]

    def update_plot(angle=0.1, noise_level=0.1):
        # 2. 混合 (Mixing)
        theta = angle * np.pi
        # 回転行列
        R = np.array([[np.cos(theta), -np.sin(theta)], 
                      [np.sin(theta),  np.cos(theta)]])
        # スケーリング
        Scale = np.array([[1, 0], [0, 2]]) 
        A = R @ Scale
        
        # 信号の混合
        X = np.dot(S, A.T)
        
        # ノイズ付加
        if noise_level > 0:
            X += noise_level * np.random.normal(size=X.shape)

        # 3. PCA
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X)
        
        # 4. 因子分析 (Factor Analysis)
        fa = FactorAnalysis(n_components=2, rotation='varimax')
        X_fa = fa.fit_transform(X)

        # 5. ICA
        # whiten='unit-variance' is required for newer sklearn versions
        ica = FastICA(n_components=2, random_state=0, whiten='unit-variance')
        X_ica = ica.fit_transform(X)

        # --- 可視化 ---
        fig = plt.figure(figsize=(15, 10))
        
        # レイアウト (2行3列)
        # 上段：原信号、混合信号、(空き)
        # 下段：PCA, FA, ICA
        
        ax1 = plt.subplot(2, 3, 1)
        ax1.plot(S[:, 0], alpha=0.7, label='Sine')
        ax1.plot(S[:, 1], alpha=0.7, label='Sawtooth')
        ax1.set_title('1. 原信号 (True Sources)')
        ax1.legend()

        ax2 = plt.subplot(2, 3, 2)
        ax2.scatter(X[:, 0], X[:, 1], alpha=0.5, s=1)
        ax2.set_title(f'2. 混合信号 (Mixed)\nAngle={angle:.2f} $\pi$')
        ax2.set_xlabel('x1')
        ax2.set_ylabel('x2')
        ax2.axis('equal')

        ax3 = plt.subplot(2, 3, 4)
        ax3.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5, s=1)
        ax3.set_title('3. PCA結果 (分散最大化)')
        ax3.set_xlabel('PC1')
        ax3.set_ylabel('PC2')
        ax3.axis('equal')
        
        ax4 = plt.subplot(2, 3, 5)
        ax4.scatter(X_fa[:, 0], X_fa[:, 1], alpha=0.5, s=1)
        ax4.set_title('4. 因子分析 (FA) 結果')
        ax4.set_xlabel('Factor1')
        ax4.set_ylabel('Factor2')
        ax4.axis('equal')

        ax5 = plt.subplot(2, 3, 6)
        ax5.plot(X_ica[:, 0], alpha=0.7, label='IC1')
        ax5.plot(X_ica[:, 1], alpha=0.7, label='IC2')
        ax5.set_title('5. ICA結果 (独立性最大化)')
        ax5.legend()
        
        plt.tight_layout()
        plt.show()

    style = {'description_width': 'initial'}
    widgets.interact(update_plot, 
                     angle=widgets.FloatSlider(min=0, max=1.0, step=0.05, value=0.1, description='Mixing Angle (x $\pi$):', style=style),
                     noise_level=widgets.FloatSlider(min=0, max=1.0, step=0.1, value=0.1, description='Noise Level:', style=style)
                    )

run_interactive_demo()